# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import json
import pandas as pd
import numpy as np
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
   # print(filepath)
    all_files = []
    for root, dirs, files in os.walk(filepath):
        print (root, dirs,files)
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            print(f)
            all_files.append(os.path.abspath(f))
            return all_files
        num_files = len(all_files)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = '/home/workspace/data/song_data'
song_files = get_files(filepath)

/home/workspace/data/song_data ['A', '.ipynb_checkpoints'] []
/home/workspace/data/song_data/A ['A', '.ipynb_checkpoints', 'B'] []
/home/workspace/data/song_data/A/A ['A', '.ipynb_checkpoints', 'B', 'C'] []
/home/workspace/data/song_data/A/A/A ['.ipynb_checkpoints'] ['TRAAAMQ128F1460CD3.json', 'TRAAAMO128F1481E7F.json', 'TRAAABD128F429CF47.json', 'TRAAARJ128F9320760.json', 'TRAAAVG12903CFA543.json', 'TRAAAVO128F93133D4.json', 'TRAAADZ128F9348C2E.json', 'TRAAAPK128E0786D96.json', 'TRAAAEF128F4273421.json', 'TRAAAAW128F429D538.json', 'TRAAAFD128F92F423A.json']
/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json


In [5]:
song_files
df=pd.read_json('/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', lines = True)
print(df.head())

            artist_id  artist_latitude artist_location  artist_longitude  \
0  AR8IEZO1187B99055E              NaN                               NaN   

    artist_name   duration  num_songs             song_id          title  year  
0  Marc Shaiman  149.86404          1  SOINLJW12A8C13314C  City Slickers  2008  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
selcolumns = df[['song_id', 'title', 'artist_id', 'year','duration']].values
song_data=tuple(selcolumns.ravel().tolist())
print(song_data)

('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404)


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
from sql_queries import song_table_insert
cur.execute(song_table_insert, song_data)
print("Record inserted successfully")
conn.commit()

try: 
    cur.execute ("Select * from songs")
except psycopg2.Error as e: 
    print("Error: Issue selecting table")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()



Record inserted successfully
('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', '2008', 149.86404)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
from sql_queries import artist_table_insert
artistColumns = df[['artist_id', 'artist_name', 'artist_location','artist_latitude', 'artist_longitude']].values
artist_data = tuple(artistColumns.ravel().tolist())

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
cur.execute(artist_table_insert, artist_data)
print("Record inserted successfully")
conn.commit()

try: 
    cur.execute ("Select * from artists")
except psycopg2.Error as e: 
    print("Error: Issue selecting table")
    print (e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


Record inserted successfully
('AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
filepath = '/home/workspace/data/log_data/2018/11/'
FirstFile = os.listdir(filepath)[0]
df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-02-events.json', lines = True)
df.iloc[0].head()
print(df.iloc[0].head())

artist           N.E.R.D. FEATURING MALICE
auth                             Logged In
firstName                           Jayden
gender                                   M
itemInSession                            0
Name: 0, dtype: object


In [11]:
df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-02-events.json', lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
import pandas as pd
import calendar
import datetime
import numpy as np
from datetime import datetime
df['date'] = pd.to_datetime(df['ts'].iloc[0])
df['starttime'] = df['date'].dt.time
df['hour'] = df['date'].dt.week
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.week
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['weekday'] = df['date'].dt.week

d = {'starttime':df['starttime'], 'hour':df['hour'], 'day':df['day'], 'week': df['week'], 'month': df['month'], 'year': df['year'], 'weekday': df['weekday'] }
time_df = pd.DataFrame(data = d)

In [13]:
time_df.head()

,starttime,hour,day,week,month,year,weekday
0,00:25:41.121934,1,1,1,1,1970,1
1,00:25:41.121934,1,1,1,1,1970,1
2,00:25:41.121934,1,1,1,1,1970,1
3,00:25:41.121934,1,1,1,1,1970,1
4,00:25:41.121934,1,1,1,1,1970,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [14]:
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert, list(row))
    except psycopg2.Error as e:
        print("Error: Record not inserted")

try: 
    cur.execute ("Select * from time fetch first 10 rows only")
except psycopg2.Error as e: 
    print("Error: Issue selecting table")
    print (e)
row = cur.fetchone()
while row:
    print (row)
    row = cur.fetchone()

conn.commit()

('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')
('00:25:41.121934', 1, 1, '1', 1, 1970, '1')


In [15]:
#%debug magic

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table

- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [16]:
from sql_queries import user_table_insert
seluserscolumns = df[['userId', 'firstName', 'lastName', 'gender', 'level']].values
user_data=tuple(seluserscolumns.ravel().tolist())

d = {'userId':df['userId'], 'firstName':df['firstName'], 'lastName':df['lastName'], 'gender': df['gender'], 'level': df['level']}
user_df = pd.DataFrame(data = d)


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [17]:
for i, row in user_df.iterrows():
    try:
        cur.execute(user_table_insert, list(row))
    except psycopg2.Error as e:
        print("Error: Record not inserted" + str(row))

try: 
    cur.execute ("Select * from users fetch first 10 rows only")
except psycopg2.Error as e: 
    print("Error: Issue selecting table")
    print (e)
row = cur.fetchone()
while row:
    row = cur.fetchone()

conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [18]:
from sql_queries import songplay_table_insert, song_select
selsongplaycolumns = df[['starttime','userId', 'level', 'sessionId','location','userAgent']].values
songplay_data=tuple(selsongplaycolumns.ravel().tolist())
d = {'starttime':df['starttime'], 'userId':df['userId'], 'level':df['level'], 'sessionId': df['sessionId'], 'location': df['location'], 'userAgent': df['userAgent']}
songplay_df = pd.DataFrame(data = d)

#insert songplay record

for index, row in songplay_df.iterrows():
    try:
        cur.execute(songplay_table_insert, list(row))
    except psycopg2.Error as e:
        print("Error record not inserted")
        print(e)


try: 
    cur.execute ("Select * from songplays fetch first 5 rows only")
except psycopg2.Error as e: 
    print("Error: Issue selecting table")
    print (e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

In [ ]:
from sql_queries import song_select
df=pd.read_json('/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', lines = True)
selsong_artist_columns = df[['title', 'artist_name', 'duration']].values
selsong_artist_data=tuple(selsong_artist_columns.ravel().tolist())
d = {'title':df['title'], 'artist_name':df['artist_name'], 'duration':df['duration']}
selsong_artist_df = pd.DataFrame(data = d)

#song_select = "Select a.song_id, b.artist_id from songs a join artists b on a.artist_id = b.artist_id  where a.title = %s and artist_name = %s fetch first 5 rows only"# and duration = %s"

#get songid and artistid from song and artist tables

for index, row in selsong_artist_df.iterrows():
    try:
        cur.execute(song_select, (list(row)))
    except psycopg2.Error as e:
        print("Error: Issue selecting table")
        print (e)

        row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
 
    #(row.title, row.artist_name, row.duration))


# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.